In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import statsmodels as sm
from scipy.stats import norm

In [2]:
control_df = pd.read_csv('./../../../Datasets/UKEssen/NSCLC_Non-Pembro_GENIE_13-05-2025.csv')
treat_df = pd.read_csv('./../../../Datasets/UKEssen/NSCLC_Pembro_MSKCHORD_13-05-2025.csv')


In [3]:
control_df

,Unnamed: 0,PATIENT_ID,Group,Regimen,CLINICAL_GROUP,Sex,Race,Ethnicity,Current Age,ICD-O Histology Description,...,TUMOR_SITE_Bone,TUMOR_SITE_CNS/Brain,TUMOR_SITE_Thorax,TUMOR_SITE_Other,OS,OS_observed,TTNT,TTNT_observed,PFS,PFS_observed
0,0,GENIE-DFCI-000013,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Stage IV,Male,White,Non-Hispanic White,80,Adenocarcinoma,...,1,0,1,0,284,1,284,1,253.0,1.0
1,1,GENIE-DFCI-000136,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Stage IV,Male,White,Non-Hispanic White,69,Adenocarcinoma,...,0,1,0,0,703,1,703,1,58.0,1.0
2,2,GENIE-DFCI-000215,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Stage IV,Male,White,Non-Hispanic White,76,Adenocarcinoma,...,1,0,0,0,142,1,63,1,63.0,0.0
3,3,GENIE-DFCI-000260,Non-Pembrolizumab,Erlotinib Hydrochloride,Stage IV,Female,Other Asian,"AAAPI (Asian, Asian American, and Pacific Isla...",50,Adenocarcinoma,...,0,0,0,1,1086,1,430,1,362.0,1.0
4,4,GENIE-DFCI-000371,Non-Pembrolizumab,"Bevacizumab, Carboplatin, Pemetrexed Disodium",Stage IV,Female,White,Non-Hispanic White,65,NaN,...,0,0,0,0,956,1,956,1,956.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,601,GENIE-VICC-926041,Non-Pembrolizumab,"Investigational Drug, Investigational Drug",Stage IV,Male,White,Non-Hispanic White,71,Adenocarcinoma,...,1,1,1,1,87,1,71,1,36.0,1.0
602,602,GENIE-VICC-969019,Non-Pembrolizumab,"Bevacizumab, Carboplatin, Pemetrexed Disodium",Stage IV,Male,White,Non-Hispanic White,69,Adenocarcinoma,...,0,0,1,0,98,1,35,1,35.0,0.0
603,603,GENIE-VICC-983361,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Stage IV,Male,White,Non-Hispanic White,62,Adenocarcinoma,...,0,0,1,0,1435,0,167,1,164.0,1.0
604,604,GENIE-VICC-987046,Non-Pembrolizumab,Erlotinib Hydrochloride,Stage IV,Female,White,Non-Hispanic White,72,Adenocarcinoma,...,0,0,1,0,577,1,577,1,469.0,1.0


In [4]:
treat_df = treat_df.rename(columns={'TTF_Pembro': 'TTF',
    'TTF_Pembro_observed': 'TTF_observed',
    'PFS_Pembro': 'PFS',
    'PFS_Pembro_observed': 'PFS_observed',
    'OS_Pembro': 'OS',
    'OS_Pembro_observed': 'OS_observed'
})
merged_df = pd.concat([control_df, treat_df], ignore_index=True, sort=False)
merged_df = merged_df.rename(columns={'ICD-O Histology Description': 'Histopathology', 
                  'History for Positive PD-L1 (NLP)':'Pos_PD-L1',
                  'Smoking History (NLP)': 'Smoker'})

merged_df = merged_df.drop('Ethnicity', axis=1)

In [ ]:
histo = set(list(merged_df['Histopathology']))

for h in histo:
    print(h)

Squamous cell
Small Cell Carcinoma, Nos
Mucinous Adenocarcinoma
Non-Small Cell Carcinoma (C34._)
nan
Pseudosarcomatous Carcinoma
Large Cell Neuroendocrine Carcinoma
Acinar Cell Carcinoma
Mucin-Producing Adenocarinoma
Bronchiolo-Alveolar Carcinoma, Mucinous (C34._)
Giant Cell & Spindle Cell Carcinoma
Pleomorphic Carcinoma
Acinar Cell Cystadenocarcinoma
Sq Cell Ca, Keratinizing Type, Nos
Carcinoma
Bronchiolo-Alveolar Ca Mixed Muc/Non Mucinous (C34._)
Sq Cell Ca, Large Cell, Non-Keratinizing
Solid Carcinoma, Nos
Adenocarcinoma Combined With Other Types Of Carcinoma
Papillary Adenocarcinoma, Nos
Adenocarcinoma, Nos
Adenosquamous Carcinoma
Carcinoma, Nos
Other histologies/mixed tumor
Adenocarcinoma
Neuroendocrine Carcinoma
Small cell carcinoma
Squamous Cell Carcinoma, Nos


In [6]:
merged_df['Histopathology'] = merged_df['Histopathology'].replace({
    'Squamous Cell Carcinoma, Nos': 'Squamous Cell',
    'Small cell carcinoma': 'Small Cell Carcinoma, Nos',
    'Carcinoma, Nos': 'Carcinoma',
    'Adenocarcinoma, Nos': 'Adenocarcinoma',
    'Papillary Adenocarcinoma, Nos': 'Adenocarcinoma',
    'Mucin-Producing Adenocarinoma': 'Mucinous Adenocarcinoma',
    'Bronchiolo-Alveolar Ca Mixed Muc/Non Mucinous (C34._)': 'Bronchiolo-Alveolar Carcinoma, Mucinous (C34._)',
    'Squamous Cell Carcinoma, Nos': 'Squamous cell',
})

In [7]:
histo = set(list(merged_df['Histopathology']))

for h in histo:
    print(h)

Squamous cell
Small Cell Carcinoma, Nos
Mucinous Adenocarcinoma
Non-Small Cell Carcinoma (C34._)
nan
Pseudosarcomatous Carcinoma
Large Cell Neuroendocrine Carcinoma
Acinar Cell Carcinoma
Bronchiolo-Alveolar Carcinoma, Mucinous (C34._)
Giant Cell & Spindle Cell Carcinoma
Pleomorphic Carcinoma
Acinar Cell Cystadenocarcinoma
Sq Cell Ca, Keratinizing Type, Nos
Carcinoma
Sq Cell Ca, Large Cell, Non-Keratinizing
Solid Carcinoma, Nos
Adenocarcinoma Combined With Other Types Of Carcinoma
Adenosquamous Carcinoma
Other histologies/mixed tumor
Adenocarcinoma
Neuroendocrine Carcinoma


In [ ]:
# # Apply group_tumor_site to each row in merged_df to create a new column 'Tumor_Site_Grouped'
# def get_tumor_site_group(row):
#     # Check each tumor site column for presence (value == 1 or True)
#     if row.get('TUMOR_SITE_Liver', 0) == 1 or row.get('TUMOR_SITE_Intra-Abdominal', 0) == 1 or row.get('TUMOR_SITE_Thorax', 0) == 1:
#         return 'Visceral'
#     elif row.get('TUMOR_SITE_Bone', 0) == 1:
#         return 'Skeletal'
#     elif row.get('TUMOR_SITE_CNS/Brain', 0) == 1:
#         return 'CNS'
#     else:
#         return 'Other'

# merged_df['Tumor_Site_Grouped'] = merged_df.apply(get_tumor_site_group, axis=1)

In [8]:
# Apply group_tumor_site to each row in merged_df to create a new column 'Tumor_Site_Grouped'
def get_tumor_site(row):
    # Check each tumor site column for presence (value == 1 or True)
    if row.get('TUMOR_SITE_Liver', 0) == 1:
        return 'Liver'
    elif row.get('TUMOR_SITE_Intra-Abdominal', 0) == 1:
        return 'Abdominal'
    elif row.get('TUMOR_SITE_Thorax', 0) == 1:
        return 'Thorax'
    elif row.get('TUMOR_SITE_Bone', 0) == 1:
        return 'Skeletal'
    elif row.get('TUMOR_SITE_CNS/Brain', 0) == 1:
        return 'CNS'
    else:
        return 'Other'

merged_df['Tumor_Site'] = merged_df.apply(get_tumor_site, axis=1)

In [9]:
# print(np.unique(merged_df['Tumor_Site_Grouped'], return_counts=True))
print(np.unique(merged_df['Tumor_Site'], return_counts=True))

(array(['Abdominal', 'CNS', 'Liver', 'Other', 'Skeletal', 'Thorax'],
      dtype=object), array([234,  66, 222, 126, 111, 652]))


In [10]:
np.unique(merged_df['Regimen'])

array(['ALECTINIB, CARBOPLATIN, PACLITAXEL, PEMBROLIZUMAB, THIOTEPA',
       'Afatinib Dimaleate', 'Alectinib', 'Alectinib, Bevacizumab',
       'BEVACIZUMAB, CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED',
       'Bevacizumab, Carboplatin, Paclitaxel',
       'Bevacizumab, Carboplatin, Pemetrexed Disodium',
       'Bevacizumab, Cisplatin, Pemetrexed Disodium',
       'Bevacizumab, Erlotinib Hydrochloride',
       'Bevacizumab, Icotinib, Other antineoplastic, Pemetrexed Disodium',
       'Bevacizumab, Nabpaclitaxel, Pemetrexed Disodium',
       'Bevacizumab, Paclitaxel, Pemetrexed Disodium',
       'Bevacizumab, Pemetrexed Disodium',
       'CARBOPLATIN, CISPLATIN, PEMBROLIZUMAB, PEMETREXED',
       'CARBOPLATIN, CRIZOTINIB, PEMBROLIZUMAB, PEMETREXED',
       'CARBOPLATIN, DABRAFENIB, PEMBROLIZUMAB, PEMETREXED, TRAMETINIB',
       'CARBOPLATIN, PACLITAXEL PROTEIN-BOUND, PEMBROLIZUMAB',
       'CARBOPLATIN, PACLITAXEL PROTEIN-BOUND, PEMBROLIZUMAB, PEMETREXED',
       'CARBOPLATIN, PACLITAXEL, 

In [ ]:
def group_medication_coarse(med):

    # this is wrong because it misses the combinations
    med = str(med).upper()
    if ',' in med:
        med = med.split(',')[0].strip()

    print('med', med)
    
    # chemo = {
    #     'CARBOPLATIN', 'CISPLATIN', 'OXALIPLATIN', 'DOCETAXEL',
    #     'PACLITAXEL', 'NABPACLITAXEL', 'PACLITAXEL PROTEIN-BOUND',
    #     'PEMETREXED', 'PEMETREXED DISODIUM', 'ETOPOSIDE', 'GEMCITABINE',
    #     'GEMCITABINE HYDROCHLORIDE', 'VINORELBINE TARTRATE', 'FLUOROURACIL', 'THIOTEPA'
    # }
    
    # targeted = {
    #     'AFATINIB DIMALEATE', 'ERLOTINIB HYDROCHLORIDE', 'GEFITINIB', 'ICOTINIB',
    #     'OSIMERTINIB', 'CRIZOTINIB', 'ALECTINIB', 'DABRAFENIB',
    #     'TRAMETINIB', 'CABOZANTINIB SMALATE'
    # }
    
    # immuno = {'PEMBROLIZUMAB'}
    
    # antibodies = {'CETUXIMAB', 'NECITUMUMAB', 'BEVACIZUMAB'}
    
    # other = {'INVESTIGATIONAL DRUG', 'OTHER ANTINEOPLASTIC', 'OTHER NOS'}
    

    # if med in chemo:
    #     return 'Chemotherapy'
    # elif med in targeted:
    #     return 'Targeted Therapy'
    # elif med in immuno:
    #     return 'Immunotherapy/Pembro'
    # elif med in antibodies:
    #     return 'Monoclonal Antibody'
    # elif med in other:
    #     return 'Other/Unknown'
    # else:
    #     return 'Unclassified'


In [11]:
treatments, counts = np.unique(merged_df['Regimen'], return_counts=True)
all_treatments = []

for treatment in treatments:
    substances = treatment.split(',')
    for s in substances:
        s = s.lower()
        s = s.strip(' ')
        all_treatments.append(s.lower())

all_treatments = list(set(all_treatments))

recode = dict()
for t in all_treatments:
    for t2 in all_treatments:
        if t != t2:
             if (t2 in t):
                recode[t] = t2

In [13]:
for i in range(len(all_treatments)):
    if all_treatments[i] in recode.keys():
        all_treatments[i] = recode[all_treatments[i]]

In [14]:
regimen = merged_df['Regimen'].copy().to_numpy()
cleaned_regimen = []
for r in regimen:
    substances = r.split(',')
    for i in range(len(substances)):
        substances[i] = substances[i].strip(' ').lower()
        # check if replacement is necessary
        if substances[i] in recode.keys():
            substances[i] = recode[substances[i]]
    substances = list(set(substances))
    cleaned_string = ', '.join(substances)
    cleaned_regimen.append(cleaned_string)

In [15]:
merged_df['Cleaned_Regimen'] = cleaned_regimen

In [16]:
sorted_indices = np.argsort(counts)
for i in range (len(sorted_indices) - 1, 0, -1):
    idx = sorted_indices[i]
    print(treatments[idx], counts[idx])

CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED 381
PEMBROLIZUMAB 288
Carboplatin, Pemetrexed Disodium 143
Erlotinib Hydrochloride 82
Bevacizumab, Carboplatin, Pemetrexed Disodium 63
Cisplatin, Pemetrexed Disodium 45
CARBOPLATIN, PACLITAXEL, PEMBROLIZUMAB 41
Investigational Drug, Investigational Drug 36
CISPLATIN, PEMBROLIZUMAB, PEMETREXED 35
Bevacizumab, Cisplatin, Pemetrexed Disodium 31
CARBOPLATIN, PACLITAXEL PROTEIN-BOUND, PEMBROLIZUMAB 31
Investigational Drug 30
Carboplatin, Paclitaxel 27
Crizotinib 23
PEMBROLIZUMAB, PEMETREXED 15
Carboplatin, Gemcitabine Hydrochloride 12
Bevacizumab, Carboplatin, Paclitaxel 11
Afatinib Dimaleate 11
Carboplatin, Nabpaclitaxel 9
Osimertinib 8
Cisplatin, Gemcitabine Hydrochloride 7
Gemcitabine Hydrochloride, Vinorelbine Tartrate 4
Cisplatin, Docetaxel 4
Cisplatin, Investigational Drug, Pemetrexed Disodium 4
Cisplatin, Etoposide 4
Carboplatin, Docetaxel 3
Carboplatin, Investigational Drug, Pemetrexed Disodium 3
Pemetrexed Disodium 3
Alectinib 3
Vinorelbine Ta

In [33]:
subset_df

,Unnamed: 0,Group,Regimen,Sex,Race,Current Age,Histopathology,Smoker,Pos_PD-L1,OS,PFS,PFS_observed,Tumor_Site,Cleaned_Regimen,Grouped_Race
0,0,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,80,Adenocarcinoma,Former/Current Smoker,Yes,284.0,253.0,1.0,Liver,"pemetrexed, carboplatin",white
1,1,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,69,Adenocarcinoma,Former/Current Smoker,NaN,703.0,58.0,1.0,CNS,"pemetrexed, carboplatin",white
2,2,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,76,Adenocarcinoma,Former/Current Smoker,NaN,142.0,63.0,0.0,Skeletal,"pemetrexed, carboplatin",white
5,5,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",female,White,70,Other histologies/mixed tumor,Former/Current Smoker,Yes,571.0,95.0,1.0,Other,"pemetrexed, carboplatin",white
6,6,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,77,Adenocarcinoma,Former/Current Smoker,No,152.0,112.0,1.0,Skeletal,"pemetrexed, carboplatin",white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,800,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",female,WHITE,69,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,231.0,152.0,0.0,Thorax,"pembrolizumab, pemetrexed, carboplatin",white
1407,801,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",male,WHITE,48,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,281.0,258.0,1.0,Thorax,"pembrolizumab, pemetrexed, carboplatin",white
1408,802,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",male,WHITE,75,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,245.0,35.0,1.0,Liver,"pembrolizumab, pemetrexed, carboplatin",white
1409,803,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",female,WHITE,79,"Adenocarcinoma, Nos",Former/Current Smoker,NaN,257.0,251.0,1.0,Thorax,"pembrolizumab, pemetrexed, carboplatin",white


In [17]:
subset_df = merged_df[
    (merged_df['Regimen'] == 'CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED') |
    (merged_df['Regimen'] == 'Carboplatin, Pemetrexed Disodium')
].copy()

print(np.unique(subset_df['Regimen'], return_counts=True))

(array(['CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED',
       'Carboplatin, Pemetrexed Disodium'], dtype=object), array([381, 143]))


In [18]:
subset_df.shape

(524, 29)

In [19]:
drop_columns = ['TUMOR_SITE_Liver', 'TUMOR_SITE_Intra-Abdominal', 'TUMOR_SITE_Thorax',
                'TUMOR_SITE_Bone', 'TUMOR_SITE_CNS/Brain',
                'TUMOR_SITE_Other', 'Pembrolizumab_mono', 'CLINICAL_GROUP', 'PATIENT_ID', 
                'OS_observed', 'TTNT', 'TTNT_observed', 'ECOG', 'TTF', 'TTF_observed']
subset_df = subset_df.drop(columns=drop_columns)

In [20]:
subset_df

,Unnamed: 0,Group,Regimen,Sex,Race,Current Age,Histopathology,Smoker,Pos_PD-L1,OS,PFS,PFS_observed,Tumor_Site,Cleaned_Regimen
0,0,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Male,White,80,Adenocarcinoma,Former/Current Smoker,Yes,284.0,253.0,1.0,Liver,"pemetrexed, carboplatin"
1,1,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Male,White,69,Adenocarcinoma,Former/Current Smoker,NaN,703.0,58.0,1.0,CNS,"pemetrexed, carboplatin"
2,2,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Male,White,76,Adenocarcinoma,Former/Current Smoker,NaN,142.0,63.0,0.0,Skeletal,"pemetrexed, carboplatin"
5,5,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Female,White,70,Other histologies/mixed tumor,Former/Current Smoker,Yes,571.0,95.0,1.0,Other,"pemetrexed, carboplatin"
6,6,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",Male,White,77,Adenocarcinoma,Former/Current Smoker,No,152.0,112.0,1.0,Skeletal,"pemetrexed, carboplatin"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,800,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",FEMALE,WHITE,69,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,231.0,152.0,0.0,Thorax,"pembrolizumab, pemetrexed, carboplatin"
1407,801,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",MALE,WHITE,48,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,281.0,258.0,1.0,Thorax,"pembrolizumab, pemetrexed, carboplatin"
1408,802,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",MALE,WHITE,75,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,245.0,35.0,1.0,Liver,"pembrolizumab, pemetrexed, carboplatin"
1409,803,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",FEMALE,WHITE,79,"Adenocarcinoma, Nos",Former/Current Smoker,NaN,257.0,251.0,1.0,Thorax,"pembrolizumab, pemetrexed, carboplatin"


In [21]:
cleaned_treat, cleaned_counts = np.unique(subset_df['Cleaned_Regimen'], return_counts=True)
sorted_cleaned_indices = np.argsort(cleaned_counts)
# for i in range (len(sorted_cleaned_indices) - 1, 0, -1):
    # idx = sorted_cleaned_indices[i]
    # print(idx, cleaned_treat[idx], cleaned_counts[idx])

In [ ]:
# vals_to_drop = []
# for i in range(len(cleaned_treat) - 6, 0, -1):
#     vals_to_drop.append(cleaned_treat[sorted_cleaned_indices[i]])

In [ ]:
# merged_df['Cleaned_Regimen'].isin(vals_to_drop)

In [ ]:
# reduced_merged_df = merged_df[~merged_df['Cleaned_Regimen'].isin(vals_to_drop)].copy()

In [ ]:
# np.unique(reduced_merged_df['Cleaned_Regimen'])

In [22]:
race_replacement = {'asian-far east/indian subcont': 'POC', 
                    'american indian, aleutian, or eskimo': 'POC', 
                    'asian-far east/indian subcont': 'POC',
                    'chinese': 'POC',
                     'native hawaiian or pacific isl':'POC',
                     'no value entered': 'NA',
                     'other': 'NA',
                     'other asian': 'POC',
                     'pt refused to answer': 'NA',
                     'black': 'POC',
                     'black or african american': 'POC',
                     'unknown': 'NA'}

subset_df['Grouped_Race'] = subset_df['Race'].copy()
subset_df['Grouped_Race'] = [r.lower() for r in subset_df['Race']]
subset_df['Grouped_Race'] = subset_df['Grouped_Race'].replace(race_replacement)

In [23]:
print(np.unique(subset_df['Race'], return_counts=True))
print(np.unique(subset_df['Grouped_Race'], return_counts=True))


(array(['ASIAN-FAR EAST/INDIAN SUBCONT', 'BLACK OR AFRICAN AMERICAN',
       'Black', 'Chinese', 'OTHER', 'Other', 'Other Asian',
       'PT REFUSED TO ANSWER', 'UNKNOWN', 'Unknown', 'WHITE', 'White'],
      dtype=object), array([ 28,  21,   9,   2,  12,   3,   1,   2,   6,   6, 312, 122]))
(array(['NA', 'POC', 'white'], dtype=object), array([ 29,  61, 434]))


In [24]:
subset_df['Sex'] = [s.lower() for s in subset_df['Sex']]

In [25]:
subset_df

,Unnamed: 0,Group,Regimen,Sex,Race,Current Age,Histopathology,Smoker,Pos_PD-L1,OS,PFS,PFS_observed,Tumor_Site,Cleaned_Regimen,Grouped_Race
0,0,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,80,Adenocarcinoma,Former/Current Smoker,Yes,284.0,253.0,1.0,Liver,"pemetrexed, carboplatin",white
1,1,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,69,Adenocarcinoma,Former/Current Smoker,NaN,703.0,58.0,1.0,CNS,"pemetrexed, carboplatin",white
2,2,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,76,Adenocarcinoma,Former/Current Smoker,NaN,142.0,63.0,0.0,Skeletal,"pemetrexed, carboplatin",white
5,5,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",female,White,70,Other histologies/mixed tumor,Former/Current Smoker,Yes,571.0,95.0,1.0,Other,"pemetrexed, carboplatin",white
6,6,Non-Pembrolizumab,"Carboplatin, Pemetrexed Disodium",male,White,77,Adenocarcinoma,Former/Current Smoker,No,152.0,112.0,1.0,Skeletal,"pemetrexed, carboplatin",white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,800,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",female,WHITE,69,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,231.0,152.0,0.0,Thorax,"pembrolizumab, pemetrexed, carboplatin",white
1407,801,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",male,WHITE,48,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,281.0,258.0,1.0,Thorax,"pembrolizumab, pemetrexed, carboplatin",white
1408,802,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",male,WHITE,75,"Adenocarcinoma, Nos",Former/Current Smoker,Yes,245.0,35.0,1.0,Liver,"pembrolizumab, pemetrexed, carboplatin",white
1409,803,Pembrolizumab,"CARBOPLATIN, PEMBROLIZUMAB, PEMETREXED",female,WHITE,79,"Adenocarcinoma, Nos",Former/Current Smoker,NaN,257.0,251.0,1.0,Thorax,"pembrolizumab, pemetrexed, carboplatin",white


In [31]:
histo = subset_df['Histopathology'].to_numpy()

In [32]:
histo

array(['Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Other histologies/mixed tumor', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', nan, 'Adenocarcinoma', 'Adenocarcinoma',
       'Carcinoma', 'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Carcinoma', 'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Carcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', nan, 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma', 'Carcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', 'Adenocarcinoma', 'Adenocarcinoma',
       'Adenocarcinoma', n

In [ ]:

# Step 1: Define covariates and treatment
covariate_cols = ['Current Age', 'Sex', 'Grouped_Race'] 
categorical_cols = ['Sex', 'Grouped_Race']  # columns to one-hot encode
numeric_cols = list(set(covariate_cols) - set(categorical_cols))

X = reduced_merged_df[covariate_cols]
T = reduced_merged_df['Cleaned_Regimen']

# Step 2: Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)

# Step 3: Fit logistic regression pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', LogisticRegression(max_iter=1000))
])

pipeline.fit(X, T)

# Step 4: Predict propensity scores
reduced_merged_df['propensity_score'] = pipeline.predict_proba(X)[:, 1]

In [ ]:
reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed, pembrolizumab']['propensity_score']

In [ ]:
sns.histplot(reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed, pembrolizumab']['propensity_score'], color='blue', label='CP + Pembro', kde=True)
sns.histplot(reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed']['propensity_score'], color='red', label='CP', kde=True)
plt.title("Propensity Score Distribution by Treatment Group")
plt.xlabel("Propensity Score")
plt.ylabel("Count")
plt.legend()
plt.show()


In [ ]:
reduced_merged_df.columns

In [ ]:
sns.histplot(reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed, pembrolizumab']['propensity_score'], color='blue', label='CP + Pembro', kde=True)
sns.histplot(reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed']['propensity_score'], color='red', label='CP', kde=True)
plt.title("Propensity Score Distribution by Treatment Group")
plt.xlabel("Propensity Score")
plt.ylabel("Count")
plt.legend()
plt.show()


In [ ]:
covariate_cols = ['Tumor_Site', 'Histopathology_Grouped_Type', 'Current Age', 'Smoker'] 
categorical_cols = ['Tumor_Site', 'Histopathology_Grouped_Type', 'Smoker']  # columns to one-hot encode
numeric_cols = list(set(covariate_cols) - set(categorical_cols))

X = reduced_merged_df[covariate_cols]
T = reduced_merged_df['Cleaned_Regimen']

# Step 2: Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)

# Step 3: Fit logistic regression pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', LogisticRegression(max_iter=1000))
])

pipeline.fit(X, T)

# Step 4: Predict propensity scores
reduced_merged_df['propensity_score'] = pipeline.predict_proba(X)[:, 1]

In [ ]:
sns.histplot(reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed, pembrolizumab']['propensity_score'], color='blue', label='CP + Pembro', kde=True)
sns.histplot(reduced_merged_df[reduced_merged_df['Cleaned_Regimen'] == 'carboplatin, pemetrexed']['propensity_score'], color='red', label='CP', kde=True)
plt.title("Propensity Score Distribution by Treatment Group")
plt.xlabel("Propensity Score")
plt.ylabel("Count")
plt.legend()
plt.show()


In [ ]:
reduced_merged_df.columns

In [ ]:
# def logit_pvalue(x, p, model):
#     """ Calculate z-scores for scikit-learn LogisticRegression.
#     parameters:
#         model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
#         x:     matrix on which the model was fit
#     This function uses asymtptics for maximum likelihood estimates.
#     """
#     # p = model.predict_proba(x)
#     n = len(p)
#     m = len(model.coef_[0]) + 1
#     coefs = np.concatenate([model.intercept_, model.coef_[0]])
#     x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
#     ans = np.zeros((m, m))
#     for i in range(n):
#         ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
#     vcov = np.linalg.inv(np.matrix(ans))
#     se = np.sqrt(np.diag(vcov))
#     t =  coefs/se  
#     p = (1 - norm.cdf(abs(t))) * 2
#     return p

In [ ]:
# scores = dict()
# to_eval = ['Sex', 'Grouped_Race', 'Current Age', 'Histopathology_Grouped_Type', 'Histopathology_Grouped_Size', 'Tumor_Site_Grouped', 'Tumor_Site', 
#            'Smoker']
# categorical_cols = ['Sex', 'Grouped_Race', 'Histopathology_Grouped_Type', 'Histopathology_Grouped_Size', 'Tumor_Site_Grouped', 'Tumor_Site', 
#            'Smoker']  # columns to one-hot encode

# for i in range(len(to_eval)):
#     current = to_eval[i]
#     X = reduced_merged_df[current]
#     T = reduced_merged_df['Cleaned_Regimen']

#     if current in categorical_cols:
#         preprocessor = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
#         X_reshaped = X.values.reshape(-1, 1)
#     else:
#         preprocessor = StandardScaler()
#         X_reshaped = X.values.reshape(-1, 1)

#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('logreg', LogisticRegression())
#     ])

#     pipeline.fit(X_reshaped, T)
#     preds = pipeline.predict_proba(X_reshaped)[:, 1]

#     # Step 4: Predict propensity scores
#     scores[current] = [preds.mean(), logit_pvalue(X.reshape(-1, 1), preds, pipeline['logreg'])]

In [ ]:
import dowhy
from dowhy import CausalModel

In [ ]:
import sys
sys.path.append('./..')
from causal_discovery.mixed_test_pcmci import MixedTestPCMCI
from tigramite.data_processing import DataFrame

In [ ]:
tig_dataframe = DataFrame()

In [ ]:
# # With graph
# model=CausalModel(
#         data = df,
#         treatment=data["treatment_name"],
#         outcome=data["outcome_name"],
#         graph=data["gml_graph"],
#         instruments=data["instrument_names"]
#         )